# 🚀 Spectral Affinity: GPU-Accelerated Neural Mixer

This edition is engineered for **Extreme Performance** using **GPU Acceleration** and **Parallel Batch Processing**.

**⚡ Upgrades:**
- **GPU DSP (nnAudio):** Uses your Graphics Card (CUDA) to calculate CQT Spectrograms instantly.
- **Parallel I/O:** Loads files in background threads while the GPU crunches numbers.
- **Batch Processing:** Analyzes multiple songs simultaneously (Matrix operations).
- **AI Precision:** Uses Harmonic Pitch Class Profiles (HPCP) on the GPU for industry-leading Key detection.

---

### 1. 🏎️ Turbo-Charge Environment

In [7]:
import os
import warnings

# Suppress noise
warnings.filterwarnings('ignore')

print("⚙️ Initializing High-Performance Environment...")

# Install GPU Audio libraries
# nnAudio: GPU-accelerated spectrograms (CQT, Mel, STFT)
try:
    import nnAudio
except ImportError:
    !pip install -q nnAudio torch torchaudio librosa numpy tqdm

import torch
import torchaudio
from nnAudio.Spectrogram import CQT1992v2
import numpy as np
import librosa
from tqdm.auto import tqdm
import glob
import shutil
import re
from concurrent.futures import ThreadPoolExecutor
import gc

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 PREFERRED ACCELERATOR: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU (Slow)'}")
print(f"🚀 TORCH VERSION: {torch.__version__}")

⚙️ Initializing High-Performance Environment...
🔥 PREFERRED ACCELERATOR: Tesla T4
🚀 TORCH VERSION: 2.8.0+cu126


### 2. 🧠 GPU-Neural Analyzer Engine

In [8]:
class HarmonicBrainGPU:
    def __init__(self, device='cuda', sample_rate=22050, batch_size=8):
        self.device = device
        self.sr = sample_rate
        self.batch_size = batch_size
        
        print("🧠 Loading Neural DSP layers to VRAM...")
        # Initialize CQT (Constant-Q Transform) on GPU
        # This replaces standard FFT for musical accuracy
        self.cqt_layer = CQT1992v2(sr=self.sr, fmin=32.7, n_bins=84, bins_per_octave=12, hop_length=512)
        self.cqt_layer.to(self.device)
        
        # Define Key Profiles (Krumhansl-Schmuckler) as GPU Tensors
        self.major_profile = torch.tensor(
            [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88], 
            device=self.device
        )
        self.minor_profile = torch.tensor(
            [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17], 
            device=self.device
        )
        
        # Pre-calculate rolled profiles for all 12 keys to do matrix multiplication
        self.profiles_matrix = self._create_profile_matrix()
        
    def _create_profile_matrix(self):
        # Create a (24, 12) matrix of all key profiles (12 Major + 12 Minor)
        profiles = []
        # Majors
        for i in range(12):
            profiles.append(torch.roll(self.major_profile, i))
        # Minors
        for i in range(12):
            profiles.append(torch.roll(self.minor_profile, i))
            
        return torch.stack(profiles).t() # Tranpose for MatMul: (12, 24)

    def process_batch(self, audio_batch, durations):
        # audio_batch: List of numpy arrays
        # Stack into padded tensor
        max_len = max([len(x) for x in audio_batch])
        batch_tensor = torch.zeros(len(audio_batch), max_len, device=self.device)
        
        for i, audio in enumerate(audio_batch):
            # Convert part to tensor and copy to GPU
            # We limit analysis to first 2 minutes to save VRAM if needed, 
            # but let's try full duration for accuracy or cap at 4M samples (~3 mins)
            limit = min(len(audio), 4_000_000) 
            batch_tensor[i, :limit] = torch.from_numpy(audio[:limit]).to(self.device)

        # 1. CQT Spectrogram (GPU)
        # Output: (Batch, FreqBins, Time)
        cqt_spec = self.cqt_layer(batch_tensor)
        
        # 2. Chroma Generation (Fold bins into 12 pitch classes)
        # CQT maps frequency logarithmically, so we can reshape to (Batch, Octaves, 12, Time)
        # Then sum over octaves and time.
        # Simple approach with CQT1992v2 (84 bins = 7 octaves * 12 bins)
        batch_size, n_bins, time_steps = cqt_spec.shape
        # Reshape to (Batch, 7, 12, Time) and sum over Octaves (dim 1) and Time (dim 3)
        chroma = cqt_spec.view(batch_size, 7, 12, -1).sum(dim=(1, 3))
        
        # Normalize chroma vectors
        chroma = chroma / (chroma.norm(dim=1, keepdim=True) + 1e-6)
        
        # 3. Key Classification (Matrix Multiplication)
        # (Batch, 12) @ (12, 24) -> (Batch, 24)
        correlations = torch.matmul(chroma, self.profiles_matrix)
        best_indices = torch.argmax(correlations, dim=1).cpu().numpy()
        
        results = []
        pitch_classes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        
        for i, idx in enumerate(best_indices):
            if idx < 12:
                key = pitch_classes[idx]
                mode = 'Major'
            else:
                key = pitch_classes[idx - 12]
                mode = 'Minor'
            
            results.append({
                'key': key,
                'mode': mode,
                'duration': durations[i]
            })
            
        # Clean up VRAM
        del batch_tensor, cqt_spec, correlations
        return results
    
    def estimate_bpm_cpu(self, audio_arrays):
        # BPM is cleaner on CPU via Librosa (Temple runs are hard on GPU without complex models)
        # We run this in parallel threads alongside GPU work if possible, or usually just fast enough sequential
        bpms = []
        for y in audio_arrays:
            if len(y) > 0:
                # Use a slice for speed
                onset_env = librosa.onset.onset_strength(y=y, sr=self.sr)
                tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=self.sr)
                bpms.append(tempo[0])
            else:
                bpms.append(120.0)
        return bpms

def get_camelot_code(key, mode):
    # Standard Camelot Map
    camelot_map = {
        ('B', 'Major'): '01B', ('F#', 'Major'): '02B', ('C#', 'Major'): '03B', ('G#', 'Major'): '04B',
        ('D#', 'Major'): '05B', ('A#', 'Major'): '06B', ('F', 'Major'): '07B', ('C', 'Major'): '08B',
        ('G', 'Major'): '09B', ('D', 'Major'): '10B', ('A', 'Major'): '11B', ('E', 'Major'): '12B',
        ('G#', 'Minor'): '01A', ('D#', 'Minor'): '02A', ('A#', 'Minor'): '03A', ('F', 'Minor'): '04A',
        ('C', 'Minor'): '05A', ('G', 'Minor'): '06A', ('D', 'Minor'): '07A', ('A', 'Minor'): '08A',
        ('E', 'Minor'): '09A', ('B', 'Minor'): '10A', ('F#', 'Minor'): '11A', ('C#', 'Minor'): '12A'
    }
    # Common replacements just in case
    if key == 'Ob': key = 'C#' 
    return camelot_map.get((key, mode), "00X")

def clean_filename(filename):
    if '.' not in filename: return filename
    name_body, ext = filename.rsplit('.', 1)
    patterns = [r"^Slavic-", r"^Theme_OST-", r"^My_Workspace-", r"^audio-", r"[\(\.\-_\s]?[0-9a-fA-F]{8}-.*$"]
    for p in patterns: name_body = re.sub(p, "", name_body, flags=re.IGNORECASE)
    name_body = name_body.replace("_", " ").strip(" -(_)")
    name_body = re.sub(r"\s+", " ", name_body).strip()
    return f"{name_body if name_body else 'Unnamed'}.{ext}"

### 3. 🚦 Orchestrator (Parallel Batch I/O)

In [9]:
# --- CONFIG ---
INPUT_DIR = "/kaggle/input/datasets/danieldobles/ost-songs"
OUTPUT_DIR = "/kaggle/working/harmonic_sets_gpu"
TARGET_SET_DURATION = 60 * 60 
BATCH_SIZE = 16  # Process 16 songs at once on GPU
SAMPLE_RATE = 22050
# --------------

def load_audio_file(path):
    try:
        # Fast load, mono, fixed sample rate
        y, _ = librosa.load(path, sr=SAMPLE_RATE, mono=True)
        duration = len(y) / SAMPLE_RATE
        return y, duration, path
    except:
        return None, 0, path

print("🔍 Scanning...")
audio_extensions = ['*.mp3', '*.wav', '*.flac', '*.m4a']
file_paths = []
for ext in audio_extensions:
    file_paths.extend(glob.glob(os.path.join(INPUT_DIR, "**", ext), recursive=True))
file_paths = list(set(file_paths))
print(f"💿 Found {len(file_paths)} tracks.")

if not file_paths:
    print("❌ Empty library.")
else:
    # Initialize Brain
    brain = HarmonicBrainGPU(device=device, sample_rate=SAMPLE_RATE, batch_size=BATCH_SIZE)
    
    library_data = []
    
    # We iterate chunks of file paths
    chunks = [file_paths[i:i + BATCH_SIZE] for i in range(0, len(file_paths), BATCH_SIZE)]
    
    print(f"\n🚀 Starting GPU Batch Processing ({len(chunks)} batches)...")
    
    with ThreadPoolExecutor(max_workers=4) as loader_pool:
        # Loop through batches
        for chunk_idx, chunk_paths in enumerate(tqdm(chunks, desc="🔥 Processing Batches")):
            
            # 1. Parallel Load CPU (While GPU is busy with prev, theoretically, but here sequential loop)
            # Loading is usually the bottleneck, so we parallelize it.
            loaded_batch = list(loader_pool.map(load_audio_file, chunk_paths))
            
            # Filter failed loads
            valid_batch = [x for x in loaded_batch if x[0] is not None]
            if not valid_batch: continue
            
            audio_arrays = [x[0] for x in valid_batch]
            durations = [x[1] for x in valid_batch]
            paths = [x[2] for x in valid_batch]
            
            # 2. GPU Analysis (Key)
            key_results = brain.process_batch(audio_arrays, durations)
            
            # 3. CPU Analysis (BPM) - Fast Calc
            bpms = brain.estimate_bpm_cpu(audio_arrays)
            
            # 4. Aggregate
            for i, res in enumerate(key_results):
                camelot = get_camelot_code(res['key'], res['mode'])
                library_data.append({
                    "path": paths[i],
                    "camelot": camelot,
                    "bpm": bpms[i],
                    "duration": res['duration']
                })
            
            # Clean up RAM immediately
            del audio_arrays, loaded_batch
            if chunk_idx % 5 == 0: gc.collect()

    # --- CLUSTERING & EXPORT ---
    print("\n🎧 Clustering Mixes...")
    library_data.sort(key=lambda x: (x['camelot'], x['bpm']))
    
    if os.path.exists(OUTPUT_DIR): shutil.rmtree(OUTPUT_DIR)
    
    sets = []
    current = []
    cur_dur = 0
    
    for track in library_data:
        current.append(track)
        cur_dur += track['duration']
        if cur_dur >= TARGET_SET_DURATION:
            sets.append(current)
            current = []
            cur_dur = 0
    if current: sets.append(current)
    
    total_sets = 0
    for i, s in enumerate(sets):
        set_dir = os.path.join(OUTPUT_DIR, f"Set_{str(i+1).zfill(2)}")
        os.makedirs(set_dir, exist_ok=True)
        total_sets += 1
        
        for idx, t in enumerate(s):
            clean = clean_filename(os.path.basename(t['path']))
            new_name = f"{str(idx+1).zfill(2)} - [{t['camelot']} - {int(t['bpm'])}BPM] {clean}"
            shutil.copy2(t['path'], os.path.join(set_dir, new_name))
            
    print(f"\n✅ DONE. Generated {total_sets} sets.")
    
    # Zip & Download
    zip_name = "gpu_harmonic_mixes.zip"
    !zip -0 -rq {zip_name} harmonic_sets_gpu
    
    display(HTML(f"<h3>🚀 <a href='{zip_name}' target='_blank'>DOWNLOAD GPU MIXES</a></h3>"))
    display(HTML(f"<script>setTimeout(function(){{ document.querySelector('a[href=\"{zip_name}\"]').click(); }}, 1500);</script>"))

🔍 Scanning...
💿 Found 163 tracks.
🧠 Loading Neural DSP layers to VRAM...
CQT kernels created, time used = 0.0637 seconds

🚀 Starting GPU Batch Processing (11 batches)...


🔥 Processing Batches:   0%|          | 0/11 [00:00<?, ?it/s]


🎧 Clustering Mixes...

✅ DONE. Generated 11 sets.
